<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Regression and Classification with the Ames Housing Data

---

You have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Estimating the value of homes from fixed characteristics.

---

Your superiors have outlined this year's strategy for the company:
1. Develop an algorithm to reliably estimate the value of residential houses based on *fixed* characteristics.
2. Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Then we can use that to buy houses that are likely to sell for more than the cost of the purchase plus renovations.

Your first job is to tackle #1. You have a dataset of housing sale data with a huge amount of features identifying different aspects of the house. The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt
    
You need to build a reliable estimator for the price of the house given characteristics of the house that cannot be renovated. Some examples include:
- The neighborhood
- Square feet
- Bedrooms, bathrooms
- Basement and garage space

and many more. 

Some examples of things that **ARE renovate-able:**
- Roof and exterior features
- "Quality" metrics, such as kitchen quality
- "Condition" metrics, such as condition of garage
- Heating and electrical components

and generally anything you deem can be modified without having to undergo major construction on the house.

---

**Your goals:**
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize your model. How well does it perform? What are the best estimates of price?

> **Note:** The EDA and feature engineering component to this project is not trivial! Be sure to always think critically and creatively. Justify your actions! Use the data description file!

In [2]:
# Load the data
df_house = pd.read_csv('./housing.csv')

In [3]:
# A:
df_house.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
645,646,20,RL,NaN,10530,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2007,WD,Normal,143250
525,526,20,FV,62.0,7500,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,176000
1285,1286,50,RM,50.0,6000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2009,WD,Normal,132500
406,407,50,RL,51.0,10480,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2008,WD,Normal,115000
148,149,20,RL,63.0,7500,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Normal,141000


In [4]:
# 1. Perform any cleaning, feature engineering, and EDA you deem necessary.

In [5]:
df_house.shape

(1460, 81)

In [6]:
df_house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [7]:
# A. drop non-residential data from the DataFrame
df_house['MSZoning'].unique()

# Based on the data dictionary, these are the non-residential types in the 'MSZoning' column:
# A	Agriculture
# C	Commercial    
# FV	Floating Village Residential   
# I	Industria

array(['RL', 'RM', 'C (all)', 'FV', 'RH'], dtype=object)

In [8]:
df_house = df_house[(df_house['MSZoning'] != 'C (all)')]
df_house = df_house[(df_house['MSZoning'] != 'FV')]

In [9]:
df_house['MSZoning'].unique()

array(['RL', 'RM', 'RH'], dtype=object)

In [10]:
df_house.shape

(1385, 81)

In [11]:
# B. List all the columns to classify them into different category types like:
# quality columns
# condition columns
# heat / electric columns

df_house.columns

Index([u'Id', u'MSSubClass', u'MSZoning', u'LotFrontage', u'LotArea',
       u'Street', u'Alley', u'LotShape', u'LandContour', u'Utilities',
       u'LotConfig', u'LandSlope', u'Neighborhood', u'Condition1',
       u'Condition2', u'BldgType', u'HouseStyle', u'OverallQual',
       u'OverallCond', u'YearBuilt', u'YearRemodAdd', u'RoofStyle',
       u'RoofMatl', u'Exterior1st', u'Exterior2nd', u'MasVnrType',
       u'MasVnrArea', u'ExterQual', u'ExterCond', u'Foundation', u'BsmtQual',
       u'BsmtCond', u'BsmtExposure', u'BsmtFinType1', u'BsmtFinSF1',
       u'BsmtFinType2', u'BsmtFinSF2', u'BsmtUnfSF', u'TotalBsmtSF',
       u'Heating', u'HeatingQC', u'CentralAir', u'Electrical', u'1stFlrSF',
       u'2ndFlrSF', u'LowQualFinSF', u'GrLivArea', u'BsmtFullBath',
       u'BsmtHalfBath', u'FullBath', u'HalfBath', u'BedroomAbvGr',
       u'KitchenAbvGr', u'KitchenQual', u'TotRmsAbvGrd', u'Functional',
       u'Fireplaces', u'FireplaceQu', u'GarageType', u'GarageYrBlt',
       u'GarageFinish',

In [12]:
column_quality = ['OverallQual', 'ExterQual', 'BsmtQual', 'BsmtFinType1', 'BsmtFinType2', 'KitchenQual', 
                  'FireplaceQu', 'GarageQual', 'Fence', 'PoolQC']
column_condition = ['OverallCond', 'ExterCond', 'BsmtCond', 'GarageCond']
column_heat_elec = ['Heating', 'HeatingQC', 'CentralAir', 'Electrical']

In [13]:
#list comprehension for all other columns
column_all_others = [col for col in df_house.columns if (col not in (column_quality + 
                                                                     column_condition + 
                                                                     column_heat_elec)) ]

# check that the lenght of columns_all_others = 81 - 10 (quality) - 4 (condition) - 4 (heat/electric)
len(column_all_others)

63

In [14]:
# B.2 clean up null values in the df_house dataframe by inspecting subsetted dataframes
df_quality = df_house[column_quality]
df_quality.isnull().sum()

OverallQual        0
ExterQual          0
BsmtQual          37
BsmtFinType1      37
BsmtFinType2      38
KitchenQual        0
FireplaceQu      641
GarageQual        79
Fence           1108
PoolQC          1378
dtype: int64

In [35]:
# B.2.a Look at the basebement metric. Where there is total basement SF of 0, there should be NA in condition
# and quality metrics. Convert the NaNs into 'NA' and then subsequently use the label encoder for certain
# quality and or condition metrics for the basement to clean and ulitmately run through feature selection process

df_house[df_house['TotalBsmtSF'] == 0][['TotalBsmtSF', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
                                        'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF']].sample(5)

,TotalBsmtSF,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF
1045,0,NaN,NaN,NaN,NaN,0,NaN,0,0
17,0,NaN,NaN,NaN,NaN,0,NaN,0,0
90,0,NaN,NaN,NaN,NaN,0,NaN,0,0
532,0,NaN,NaN,NaN,NaN,0,NaN,0,0
1216,0,NaN,NaN,NaN,NaN,0,NaN,0,0


In [61]:
df_house['BsmtQual'].replace(np.nan, 'NA', inplace=True)
df_house['BsmtCond'].replace(np.nan, 'NA', inplace=True)
df_house['BsmtExposure'].replace(np.nan, 'NA', inplace=True)
df_house['BsmtFinType1'].replace(np.nan, 'NA', inplace=True)
df_house['BsmtFinType2'].replace(np.nan, 'NA', inplace=True)

In [62]:
df_house[['TotalBsmtSF', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 
          'BsmtFinSF2', 'BsmtUnfSF']].isnull().sum()

TotalBsmtSF     0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinSF1      0
BsmtFinType2    0
BsmtFinSF2      0
BsmtUnfSF       0
dtype: int64

In [69]:
# B.2.b. Look at the fireplace metrics. 
df_house['FireplaceQu'].replace(np.nan, 'NA', inplace=True)

In [70]:
df_house[['Fireplaces', 'FireplaceQu']].isnull().sum()

Fireplaces     0
FireplaceQu    0
dtype: int64

In [79]:
# B.2.c. Cleanup Garage metrics
df_house[df_house['GarageArea']==0][['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 
                                     'GarageQual', 'GarageCond']].sample(5)

,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond
826,NA,NA,NA,0,0,NA,NA
1283,NA,NA,NA,0,0,NA,NA
464,NA,NA,NA,0,0,NA,NA
163,NA,NA,NA,0,0,NA,NA
48,NA,NA,NA,0,0,NA,NA


In [80]:
df_house['GarageType'].replace(np.nan, 'NA', inplace=True)
df_house['GarageYrBlt'].replace(np.nan, 'NA', inplace=True)
df_house['GarageFinish'].replace(np.nan, 'NA', inplace=True)
df_house['GarageQual'].replace(np.nan, 'NA', inplace=True)
df_house['GarageCond'].replace(np.nan, 'NA', inplace=True)

In [82]:
# B.2.d. Cleanup Fence metrics
df_house['Fence'].replace(np.nan, 'NA', inplace=True)

In [87]:
# B.2.e. Cleanup Pool metrics
# if Pool Area = 0, then check to see if there is nan in all cases for PoolQC
df_house[df_house['PoolArea'] == 0][['PoolArea', 'PoolQC']]
df_house['PoolQC'].replace(np.nan, 'NA', inplace=True)

In [90]:
df_condition = df_house[column_condition]
df_condition.isnull().sum()

OverallCond    0
ExterCond      0
BsmtCond       0
GarageCond     0
dtype: int64

In [91]:
df_heat_elec = df_house[column_heat_elec]
df_heat_elec.isnull().sum()

Heating       0
HeatingQC     0
CentralAir    0
Electrical    1
dtype: int64

In [111]:
# B.2.f Clean up the null value out of Electrical
pd.isnull(df_house[['Electrical']]).any()

Electrical    True
dtype: bool

In [114]:
np.where(df_house['Electrical'].isnull())

(array([1306]),)

In [120]:
df_house['Electrical'].value_counts()

SBrkr    1263
FuseA      90
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [122]:
df_house.iloc[[1306]]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1379,1380,80,RL,73.0,9735,Pave,NaN,Reg,Lvl,AllPub,...,0,NA,NA,NaN,0,5,2008,WD,Normal,167500


In [121]:
#replace the null value with the S8rkr value since it is the mos requently occurring 
df_house['Electrical'].replace(np.nan, 'SBrkr', inplace=True)

In [123]:
df_all_others = df_house[column_all_others]
df_all_others.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1385 entries, 0 to 1459
Data columns (total 63 columns):
Id               1385 non-null int64
MSSubClass       1385 non-null int64
MSZoning         1385 non-null object
LotFrontage      1134 non-null float64
LotArea          1385 non-null int64
Street           1385 non-null object
Alley            65 non-null object
LotShape         1385 non-null object
LandContour      1385 non-null object
Utilities        1385 non-null object
LotConfig        1385 non-null object
LandSlope        1385 non-null object
Neighborhood     1385 non-null object
Condition1       1385 non-null object
Condition2       1385 non-null object
BldgType         1385 non-null object
HouseStyle       1385 non-null object
YearBuilt        1385 non-null int64
YearRemodAdd     1385 non-null int64
RoofStyle        1385 non-null object
RoofMatl         1385 non-null object
Exterior1st      1385 non-null object
Exterior2nd      1385 non-null object
MasVnrType       1380 non

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Determine any value of *changeable* property characteristics unexplained by the *fixed* ones.

---

Now that you have a model that estimates the price of a house based on its static characteristics, we can move forward with part 2 and 3 of the plan: what are the costs/benefits of quality, condition, and renovations?

There are two specific requirements for these estimates:
1. The estimates of effects must be in terms of dollars added or subtracted from the house value. 
2. The effects must be on the variance in price remaining from the first model.

The residuals from the first model (training and testing) represent the variance in price unexplained by the fixed characteristics. Of that variance in price remaining, how much of it can be explained by the easy-to-change aspects of the property?

---

**Your goals:**
1. Evaluate the effect in dollars of the renovate-able features. 
- How would your company use this second model and its coefficients to determine whether they should buy a property or not? Explain how the company can use the two models you have built to determine if they can make money. 
- Investigate how much of the variance in price remaining is explained by these features.
- Do you trust your model? Should it be used to evaluate which properties to buy and fix up?

In [ ]:
# A:

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. What property characteristics predict an "abnormal" sale?

---

The `SaleCondition` feature indicates the circumstances of the house sale. From the data file, we can see that the possibilities are:

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)
       
One of the executives at your company has an "in" with higher-ups at the major regional bank. His friends at the bank have made him a proposal: if he can reliably indicate what features, if any, predict "abnormal" sales (foreclosures, short sales, etc.), then in return the bank will give him first dibs on the pre-auction purchase of those properties (at a dirt-cheap price).

He has tasked you with determining (and adequately validating) which features of a property predict this type of sale. 

---

**Your task:**
1. Determine which features predict the `Abnorml` category in the `SaleCondition` feature.
- Justify your results.

This is a challenging task that tests your ability to perform classification analysis in the face of severe class imbalance. You may find that simply running a classifier on the full dataset to predict the category ends up useless: when there is bad class imbalance classifiers often tend to simply guess the majority class.

It is up to you to determine how you will tackle this problem. I recommend doing some research to find out how others have dealt with the problem in the past. Make sure to justify your solution. Don't worry about it being "the best" solution, but be rigorous.

Be sure to indicate which features are predictive (if any) and whether they are positive or negative predictors of abnormal sales.

In [ ]:
# A: